In [237]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import json
from dataloader import Twitch
import tensorflow as tf
from spektral.layers import GCNConv
from tensorflow.keras import layers
from utils import *
from spektral.data.loaders import SingleLoader
from sklearn.model_selection import train_test_split
from spektral.layers import GCNConv, GlobalMaxPool, GlobalAvgPool
from spektral.models.gcn import GCN
from spektral.transforms import LayerPreprocess, AdjToSpTensor
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [187]:
class GNNNodeClassifier(tf.keras.Model):
    def __init__(self, num_classes, hidden_channels, dropout_rate=0.2):
        super(GNNNodeClassifier, self).__init__()
        self.conv1 = GCNConv(hidden_channels, activation='relu')
        self.conv2 = GCNConv(hidden_channels, activation='relu')
        self.conv3 = GCNConv(hidden_channels, activation='relu')
        self.dropout = Dropout(dropout_rate)
        self.pool1 = GlobalMaxPool()
        self.pool2 = GlobalAvgPool()
        self.fn = layers.Dense(num_classes)
    
    def call(self, inputs):
        x, adj = inputs
        
        x = self.conv1([x, adj])
        x = self.conv2([x, adj])
        x = self.conv3([x, adj])
        #x = self.dropout(x)
        x = tf.concat([self.pool1(x), self.pool2(x)], axis=1)
        x = self.fn(x)
        
        return x

In [235]:
class BasicGCN(tf.keras.Model):
    def __init__(self, n_labels, hidden_channels):
        super(BasicGCN, self).__init__()
        self.conv1 = GCNConv(hidden_channels, activation='relu')
        self.conv2 = GCNConv(n_labels, activation='softmax')
    
    def call(self, inputs):
        x, adj = inputs
        
        x = self.conv1([x, adj])
        x = self.conv2([x, adj])
        
        return x

In [248]:
dataset = Twitch("DE", normalize_x=True, transforms=[LayerPreprocess(GCNConv), AdjToSpTensor()])
graph = dataset[0]
x, adj, y = graph.x, graph.a, graph.y

In [249]:
mask_tr, mask_va, mask_te = dataset.mask_tr, dataset.mask_va, dataset.mask_te
mask_tr

<tf.Tensor: shape=(9498,), dtype=int32, numpy=array([1, 1, 1, ..., 0, 0, 0])>

In [255]:
pred = model([x, adj])

In [259]:
y

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [1., 0.],
       [1., 0.]])

In [251]:
lr = 1e-2
epochs = 20
patience = 10

model = BasicGCN(n_labels=2, hidden_channels=64)
loss_fn = BinaryCrossentropy()
optim = Adam(learning_rate=0.01)

def train_step():
    with tf.GradientTape() as tape:
        pred = model([x, adj])
        loss = loss_fn(y[mask_tr], pred.numpy()[mask_tr])
    gradients = tape.gradient(loss, model.trainable_variables)
    optim.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

In [250]:
losses = []

for i in range(epochs):
    losses.append(train_step())

plt.plot(losses)

TypeError: Only integers, slices (`:`), ellipsis (`...`), tf.newaxis (`None`) and scalar tf.int32/tf.int64 tensors are valid indices, got <tf.Tensor: shape=(9498,), dtype=int32, numpy=array([1, 1, 1, ..., 0, 0, 0])>

In [189]:
print("Evaluating model.")
loader_te = SingleLoader(dataset, sample_weights=weights_te)
eval_results = model.evaluate(loader_te.load(), steps=loader_te.steps_per_epoch)
print("Done.\n" "Test loss: {}\n" "Test accuracy: {}".format(*eval_results))

Evaluating model.
1/1 [==============================] - 0s 200ms/step - loss: 3.6388e-04 - acc: 0.6047
Done.
Test loss: 0.00036387614090926945
Test accuracy: 0.604736864566803


In [190]:
x = next(iter(loader_tr))

In [213]:
dataset[0]

Graph(n_nodes=9498, n_node_features=3169, n_edge_features=None, n_labels=1)

In [200]:
x[0][0].shape

(9498, 3169)

In [205]:
data.mask_tr

array([1., 0., 1., ..., 1., 0., 1.])